In [142]:
import os
import sys
import pyspark
from pyspark.sql.window import *
from pyspark.sql.functions import *
import operator
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
from pyspark import SQLContext


In [212]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df = spark.read.csv("C:\\Users\\Ajibola Vincent\\Documents\\Udacity\\ay1.csv",header=True,sep=",");
sqlContext = SQLContext(spark)
df.count()


506

In [245]:
df = df.select('account_id','account_nbr').na.drop(subset=["account_nbr"])
df.printSchema()
w = Window.partitionBy('account_id').orderBy('account_nbr')

root
 |-- account_id: string (nullable = true)
 |-- account_nbr: string (nullable = true)



In [260]:
'''sd = df.withColumn('sorted_list', collect_set('account_nbr').over(w))\
        .groupBy('account_id')\
        .agg(max('sorted_list').alias('sorted_list'))'''

sd = df.groupby("account_id")\
  .agg(collect_set("account_nbr").alias('account_nbr'))\
  .orderBy("account_id")
sd.show()

+----------+--------------------+
|account_id|         account_nbr|
+----------+--------------------+
|        15|          [24265569]|
|        18|          [14567877]|
|      1844|[42988401, 8, 429...|
|         2|[89597016, 664871...|
|      2177|          [62457513]|
|        22|          [97847811]|
|        23|          [20774019]|
|      2357|          [93012204]|
|        26|[12891853, 73418420]|
|        27|          [28175699]|
|        31|          [12345679]|
|        33|          [33047658]|
|        34|[19993174, 589990...|
|        35|[99848401, 567886...|
|      3818|          [86543521]|
|       485| [4968433, 42968433]|
|         9|          [13943797]|
+----------+--------------------+



In [261]:
sd = sd.withColumn(
    'account_nbr', sd.account_nbr.getItem(0)
)
#sd = sd.orderBy(asc('account_id'))
sd = sd.withColumn("account_id", sd.account_id.cast(IntegerType()))
sd = sd.sort(asc('account_id'))
sd.printSchema()

root
 |-- account_id: integer (nullable = true)
 |-- account_nbr: string (nullable = true)



In [299]:
sd.show()
df = df.withColumn("account_id", df.account_id.cast(IntegerType()))

+----------+-----------+
|account_id|account_nbr|
+----------+-----------+
|         2|   89597016|
|         9|   13943797|
|        15|   24265569|
|        18|   14567877|
|        22|   97847811|
|        23|   20774019|
|        26|   12891853|
|        27|   28175699|
|        31|   12345679|
|        33|   33047658|
|        34|   19993174|
|        35|   99848401|
|       485|    4968433|
|      1844|   42988401|
|      2177|   62457513|
|      2357|   93012204|
|      3818|   86543521|
+----------+-----------+



In [304]:
re = df.join(sd,(df.account_nbr == sd.account_nbr) & (df.account_id == sd.account_id),how='left_semi')


In [305]:
re.count()

191

In [307]:
re.show(120)

+----------+-----------+
|account_id|account_nbr|
+----------+-----------+
|         2|   89597016|
|         2|   89597016|
|         2|   89597016|
|         2|   89597016|
|         2|   89597016|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
|         9|   13943797|
